In [24]:
with open('day19_input.txt') as file:
    puzzle_input = file.read()

blueprints = []

for line in puzzle_input.splitlines():
    words = line.split()
    integers = []
    for word in words:
        try:
            integers.append(int(word))
        except:
            pass
    blueprints.append(tuple(integers))

def calc_max_geodes_possible(geode_count, geode_robot_count, time_remaining):
    for time in range(time_remaining):
        geode_count += geode_robot_count
        geode_robot_count += 1
        
    return geode_count
    
def calc_max_geodes(blueprint, duration):
    states = {}
    max_geodes_found = 0

    def calc_max_geodes_recursive(
        ore_count,
        clay_count,
        obsidian_count,
        geode_count,
        ore_robot_count,
        clay_robot_count,
        obsidian_robot_count,
        geode_robot_count,
        ore_robot_ore_cost,
        clay_robot_ore_cost,
        obsidian_robot_ore_cost,
        obsidian_robot_clay_cost,
        geode_robot_ore_cost,
        geode_robot_obsidian_cost,
        time_remaining):
        
        nonlocal max_geodes_found
        
        if time_remaining == 0:
            max_geodes_found = max(max_geodes_found, geode_count)
            return geode_count

        if (
            ore_count,
            clay_count,
            obsidian_count,
            geode_count,
            ore_robot_count,
            clay_robot_count,
            obsidian_robot_count,
            geode_robot_count,
            time_remaining) in states:
            return states[(
                ore_count,
                clay_count,
                obsidian_count,
                geode_count,
                ore_robot_count,
                clay_robot_count,
                obsidian_robot_count,
                geode_robot_count,
                time_remaining)]

        if calc_max_geodes_possible(geode_count, geode_robot_count, time_remaining) < max_geodes_found:
            # sub-optimal
            return geode_count
        
        next_ore_count =  ore_count + ore_robot_count
        next_clay_count = clay_count + clay_robot_count
        next_obsidian_count = obsidian_count + obsidian_robot_count
        next_geode_count = geode_count + geode_robot_count

        max_geodes = [0]

        if ore_count >= ore_robot_ore_cost and ore_robot_count < max(ore_robot_ore_cost, clay_robot_ore_cost, obsidian_robot_ore_cost, geode_robot_ore_cost):
            max_geodes.append(calc_max_geodes_recursive(
                next_ore_count - ore_robot_ore_cost,
                next_clay_count,
                next_obsidian_count,
                next_geode_count,
                ore_robot_count + 1,
                clay_robot_count,
                obsidian_robot_count,
                geode_robot_count,
                ore_robot_ore_cost,
                clay_robot_ore_cost,
                obsidian_robot_ore_cost,
                obsidian_robot_clay_cost,
                geode_robot_ore_cost,
                geode_robot_obsidian_cost,
                time_remaining - 1))

        if ore_count >= clay_robot_ore_cost and clay_robot_count < obsidian_robot_clay_cost:
            max_geodes.append(calc_max_geodes_recursive(
                next_ore_count - clay_robot_ore_cost,
                next_clay_count,
                next_obsidian_count,
                next_geode_count,
                ore_robot_count,
                clay_robot_count + 1,
                obsidian_robot_count,
                geode_robot_count,
                ore_robot_ore_cost,
                clay_robot_ore_cost,
                obsidian_robot_ore_cost,
                obsidian_robot_clay_cost,
                geode_robot_ore_cost,
                geode_robot_obsidian_cost,
                time_remaining - 1))

        if ore_count >= obsidian_robot_ore_cost and clay_count >= obsidian_robot_clay_cost and obsidian_robot_count < geode_robot_obsidian_cost:
            max_geodes.append(calc_max_geodes_recursive(
                next_ore_count - obsidian_robot_ore_cost,
                next_clay_count - obsidian_robot_clay_cost,
                next_obsidian_count,
                next_geode_count,
                ore_robot_count,
                clay_robot_count,
                obsidian_robot_count + 1,
                geode_robot_count,
                ore_robot_ore_cost,
                clay_robot_ore_cost,
                obsidian_robot_ore_cost,
                obsidian_robot_clay_cost,
                geode_robot_ore_cost,
                geode_robot_obsidian_cost,
                time_remaining - 1))

        if ore_count >= geode_robot_ore_cost and obsidian_count >= geode_robot_obsidian_cost:
            max_geodes.append(calc_max_geodes_recursive(
                next_ore_count - geode_robot_ore_cost,
                next_clay_count,
                next_obsidian_count - geode_robot_obsidian_cost,
                next_geode_count,
                ore_robot_count,
                clay_robot_count,
                obsidian_robot_count,
                geode_robot_count + 1,
                ore_robot_ore_cost,
                clay_robot_ore_cost,
                obsidian_robot_ore_cost,
                obsidian_robot_clay_cost,
                geode_robot_ore_cost,
                geode_robot_obsidian_cost,
                time_remaining - 1))

        max_geodes.append(calc_max_geodes_recursive(
            next_ore_count,
            next_clay_count,
            next_obsidian_count,
            next_geode_count,
            ore_robot_count,
            clay_robot_count,
            obsidian_robot_count,
            geode_robot_count,
            ore_robot_ore_cost,
            clay_robot_ore_cost,
            obsidian_robot_ore_cost,
            obsidian_robot_clay_cost,
            geode_robot_ore_cost,
            geode_robot_obsidian_cost,
            time_remaining - 1))

        result = max(max_geodes)
        states[(
            ore_count,
            clay_count,
            obsidian_count,
            geode_count,
            ore_robot_count,
            clay_robot_count,
            obsidian_robot_count,
            geode_robot_count,
            time_remaining)] = result
        return result

    max_geodes = calc_max_geodes_recursive(0, 0, 0, 0, 1, 0, 0, 0, *blueprint, duration)
    return max_geodes

In [30]:
def calc_quality_levels(blueprints):
    return sum((n + 1) * calc_max_geodes(blueprint, 24) for n, blueprint in enumerate(blueprints))

calc_quality_levels(blueprints)

1009

In [29]:
import math

math.prod([calc_max_geodes(blueprint, 32) for blueprint in blueprints[:3]])

18816